In [170]:
!git clone https://github.com/AIP-Team9/Team9_project.git

fatal: destination path 'Team9_project' already exists and is not an empty directory.


In [171]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [172]:
file_paths = [
    '/content/Team9_project/dataset/data_2024_1.csv',
    '/content/Team9_project/dataset/data_2024_2.csv',
    '/content/Team9_project/dataset/data_2024_3.csv'
]

# 빈 리스트 생성 (각 CSV 파일의 데이터프레임을 저장할 예정)
data_frames = []

# 각 CSV 파일을 순회하면서 데이터프레임 생성 및 리스트에 추가
for file_path in file_paths:
    df = pd.read_csv(file_path, encoding='euc-kr')
    data_frames.append(df)

# 데이터프레임 합치기 (concatenate)
data = pd.concat(data_frames, ignore_index=True)

In [173]:
data_seoul = data[data['관측지점'].isin(['홍릉_도심', '홍릉_숲내부5m', '홍릉_숲외부20m','시화_산단', '시화_차단숲', '시화_주거', '양재_도로', '양재_200m', '관악_도심', '관악_캠퍼스','관악_숲'])]

display(data_seoul)

,관측지점,관측시간,관측온도,관측습도,관측기압,관측풍속,관측풍향,관측최대풍속,관측미세먼지,관측초미세먼지,...,(AVOC)배관관측온도,(BVOC)관측온도,(BVOC)관측습도,(BVOC)관측기압,(BVOC)관측풍속,(BVOC)관측풍향,(BVOC)관측최대풍속,(BVOC)관측미세먼지,(BVOC)관측초미세먼지,(BVOC)관측극초미세먼지
0,홍릉_도심,2024-01-01 00:00,1.9,91.0,1026.1,0.9,328.8,1.6,38.5,38.3,...,103.9,2.0,91.2,1026.1,0.5,344.0,1.6,12.5,12.6,12.4
1,홍릉_숲내부5m,2024-01-01 00:00,1.0,88.9,1020.1,0.2,14.4,0.6,38.6,38.0,...,105.5,1.0,89.0,1020.2,0.4,24.1,0.9,10.8,10.5,10.3
2,홍릉_숲외부20m,2024-01-01 00:00,0.9,91.3,1017.6,0.8,4.2,2.0,35.4,34.5,...,112.3,0.9,91.4,1017.6,1.1,10.8,2.0,12.6,12.0,11.8
6,시화_산단,2024-01-01 00:00,-0.9,93.5,1026.2,0.0,0.0,NaN,48.5,47.3,...,107.0,-0.9,93.4,1026.2,0.0,0.0,NaN,15.7,15.1,14.5
7,시화_차단숲,2024-01-01 00:00,1.5,85.1,1024.7,0.0,0.0,0.5,51.2,49.5,...,103.4,1.6,85.0,1024.7,0.0,2.0,0.5,15.6,14.9,14.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727187,양재_도로,2024-03-31 23:50,10.0,47.3,1010.6,0.5,149.1,NaN,56.3,21.9,...,103.1,10.0,47.3,1010.6,0.5,92.5,NaN,13.3,2.8,2.0
727188,양재_200m,2024-03-31 23:50,10.2,49.0,1010.4,0.3,134.3,0.9,23.2,15.1,...,105.5,10.2,49.1,1010.3,0.2,64.3,0.9,1.1,0.0,0.0
727189,관악_도심,2024-03-31 23:50,7.2,59.7,1006.6,1.0,123.4,2.1,47.1,19.6,...,105.2,7.1,59.8,1006.6,0.8,127.8,2.1,8.5,2.2,1.7
727190,관악_캠퍼스,2024-03-31 23:50,9.7,47.0,998.3,0.8,121.0,1.6,53.6,21.0,...,112.1,9.7,46.9,998.2,0.7,123.8,1.6,13.6,3.7,2.8


In [174]:
data_seoul["관측시간"] = pd.to_datetime(data_seoul["관측시간"], format="%Y-%m-%d %H:%M")

data_seoul[["관측지점", "관측지점세부"]] = data["관측지점"].str.split('_', expand=True)

columns_to_fill = ["관측최대풍속"]
columns_to_fill = ["관측최대풍속", "(AVOC)관측최대풍속", "(BVOC)관측최대풍속"]

for column in columns_to_fill:
    data_seoul[column] = data.groupby(["관측시간", "관측지점"])[column].transform(
        lambda x: x.fillna(x.mean())
    )

    # 그래도 남아있는 결측치는 관측지점으로 채우기
    data_seoul[column] = data.groupby('관측지점')[column].transform(
        lambda x: x.fillna(x.mean())
    )

    # 그래도 남아있는 결측치는 관측시간으로 채우기
    data_seoul[column] = data.groupby('관측시간')[column].transform(
        lambda x: x.fillna(x.mean())
    )

    # 여전히 남아있는 결측치는 전체 평균으로 채우기
    overall_mean = data_seoul[column].mean()
    data_seoul[column].fillna(overall_mean, inplace=True)

<ipython-input-174-f269e34a33d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_seoul["관측시간"] = pd.to_datetime(data_seoul["관측시간"], format="%Y-%m-%d %H:%M")
<ipython-input-174-f269e34a33d4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_seoul[["관측지점", "관측지점세부"]] = data["관측지점"].str.split('_', expand=True)
<ipython-input-174-f269e34a33d4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [175]:
seoul_time = data_seoul['관측시간']
seoul_label = data_seoul[['관측미세먼지', '관측초미세먼지']]

display(seoul_time)
display(seoul_label)
print(seoul_label.isnull().sum())

0        2024-01-01 00:00:00
1        2024-01-01 00:00:00
2        2024-01-01 00:00:00
6        2024-01-01 00:00:00
7        2024-01-01 00:00:00
                 ...        
727187   2024-03-31 23:50:00
727188   2024-03-31 23:50:00
727189   2024-03-31 23:50:00
727190   2024-03-31 23:50:00
727191   2024-03-31 23:50:00
Name: 관측시간, Length: 76184, dtype: datetime64[ns]

,관측미세먼지,관측초미세먼지
0,38.5,38.3
1,38.6,38.0
2,35.4,34.5
6,48.5,47.3
7,51.2,49.5
...,...,...
727187,56.3,21.9
727188,23.2,15.1
727189,47.1,19.6
727190,53.6,21.0


관측미세먼지     0
관측초미세먼지    0
dtype: int64


In [176]:
seoul_data = data_seoul.drop(columns=['관측지점','관측시간','관측미세먼지', '관측초미세먼지','관측지점세부'])

display(seoul_data)

,관측온도,관측습도,관측기압,관측풍속,관측풍향,관측최대풍속,관측극초미세먼지,배관관측온도,(AVOC)관측온도,(AVOC)관측습도,...,(AVOC)배관관측온도,(BVOC)관측온도,(BVOC)관측습도,(BVOC)관측기압,(BVOC)관측풍속,(BVOC)관측풍향,(BVOC)관측최대풍속,(BVOC)관측미세먼지,(BVOC)관측초미세먼지,(BVOC)관측극초미세먼지
0,1.9,91.0,1026.1,0.9,328.8,1.600000,37.6,0.4,2.0,91.3,...,103.9,2.0,91.2,1026.1,0.5,344.0,1.600000,12.5,12.6,12.4
1,1.0,88.9,1020.1,0.2,14.4,0.600000,37.3,0.4,0.9,89.1,...,105.5,1.0,89.0,1020.2,0.4,24.1,0.900000,10.8,10.5,10.3
2,0.9,91.3,1017.6,0.8,4.2,2.000000,33.8,0.2,0.9,91.4,...,112.3,0.9,91.4,1017.6,1.1,10.8,2.000000,12.6,12.0,11.8
6,-0.9,93.5,1026.2,0.0,0.0,1.225843,46.0,0.1,-0.8,93.4,...,107.0,-0.9,93.4,1026.2,0.0,0.0,1.316854,15.7,15.1,14.5
7,1.5,85.1,1024.7,0.0,0.0,0.500000,48.1,0.4,1.6,85.0,...,103.4,1.6,85.0,1024.7,0.0,2.0,0.500000,15.6,14.9,14.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727187,10.0,47.3,1010.6,0.5,149.1,1.318605,13.6,0.3,10.0,47.3,...,103.1,10.0,47.3,1010.6,0.5,92.5,1.402326,13.3,2.8,2.0
727188,10.2,49.0,1010.4,0.3,134.3,0.900000,11.6,0.4,10.1,49.2,...,105.5,10.2,49.1,1010.3,0.2,64.3,0.900000,1.1,0.0,0.0
727189,7.2,59.7,1006.6,1.0,123.4,2.100000,12.7,0.4,7.0,60.0,...,105.2,7.1,59.8,1006.6,0.8,127.8,2.100000,8.5,2.2,1.7
727190,9.7,47.0,998.3,0.8,121.0,1.600000,13.8,0.3,9.6,46.8,...,112.1,9.7,46.9,998.2,0.7,123.8,1.600000,13.6,3.7,2.8


In [177]:
import datetime as dt

seoul_time = pd.to_datetime(seoul_time)
seoul_month = seoul_time.dt.month
seoul_day = seoul_time.dt.day
seoul_hour = seoul_time.dt.hour
seoul_minute = seoul_time.dt.minute
seoul_second = seoul_time.dt.second

seoul_daytime = 30*seoul_month + seoul_day
seoul_time = 60*seoul_hour + seoul_minute


display(seoul_month)
display(seoul_day)
display(seoul_hour)
display(seoul_minute)
display(seoul_second)
display(seoul_daytime)
display(seoul_time)

0         1
1         1
2         1
6         1
7         1
         ..
727187    3
727188    3
727189    3
727190    3
727191    3
Name: 관측시간, Length: 76184, dtype: int32

0          1
1          1
2          1
6          1
7          1
          ..
727187    31
727188    31
727189    31
727190    31
727191    31
Name: 관측시간, Length: 76184, dtype: int32

0          0
1          0
2          0
6          0
7          0
          ..
727187    23
727188    23
727189    23
727190    23
727191    23
Name: 관측시간, Length: 76184, dtype: int32

0          0
1          0
2          0
6          0
7          0
          ..
727187    50
727188    50
727189    50
727190    50
727191    50
Name: 관측시간, Length: 76184, dtype: int32

0         0
1         0
2         0
6         0
7         0
         ..
727187    0
727188    0
727189    0
727190    0
727191    0
Name: 관측시간, Length: 76184, dtype: int32

0          31
1          31
2          31
6          31
7          31
         ... 
727187    121
727188    121
727189    121
727190    121
727191    121
Name: 관측시간, Length: 76184, dtype: int32

0            0
1            0
2            0
6            0
7            0
          ... 
727187    1430
727188    1430
727189    1430
727190    1430
727191    1430
Name: 관측시간, Length: 76184, dtype: int32

In [178]:
seoul_data['경과일'] = seoul_daytime
seoul_data['경과시간'] = seoul_time
seoul_data['관측미세먼지'] = seoul_label['관측미세먼지']
seoul_data['관측초미세먼지'] = seoul_label['관측초미세먼지']

display(seoul_data.isnull().sum())

관측온도              0
관측습도              0
관측기압              0
관측풍속              0
관측풍향              0
관측최대풍속            0
관측극초미세먼지          0
배관관측온도            0
(AVOC)관측온도        0
(AVOC)관측습도        0
(AVOC)관측기압        0
(AVOC)관측풍속        0
(AVOC)관측풍향        0
(AVOC)관측최대풍속      0
(AVOC)관측미세먼지      0
(AVOC)관측초미세먼지     0
(AVOC)관측극초미세먼지    0
(AVOC)배관관측온도      0
(BVOC)관측온도        0
(BVOC)관측습도        0
(BVOC)관측기압        0
(BVOC)관측풍속        0
(BVOC)관측풍향        0
(BVOC)관측최대풍속      0
(BVOC)관측미세먼지      0
(BVOC)관측초미세먼지     0
(BVOC)관측극초미세먼지    0
경과일               0
경과시간              0
관측미세먼지            0
관측초미세먼지           0
dtype: int64

In [184]:
sorted_seoul_data = seoul_data.sort_values(by=['경과일', '경과시간'])

sorted_seoul_data.to_csv("data_seoul.csv", index = False)

In [185]:
train_ratio = 0.9

train_size = int(len(sorted_seoul_data) * train_ratio)
train_data = sorted_seoul_data.iloc[:train_size]
test_data = sorted_seoul_data.iloc[train_size:]

print("훈련 데이터 크기:", len(train_data))
print("테스트 데이터 크기:", len(test_data))


train_data.to_csv("train_seoul.csv", index=False)
test_data.to_csv("test_seoul.csv", index=False)

훈련 데이터 크기: 68565
테스트 데이터 크기: 7619


In [186]:
display(train_data)
display(test_data)

,관측온도,관측습도,관측기압,관측풍속,관측풍향,관측최대풍속,관측극초미세먼지,배관관측온도,(AVOC)관측온도,(AVOC)관측습도,...,(BVOC)관측풍속,(BVOC)관측풍향,(BVOC)관측최대풍속,(BVOC)관측미세먼지,(BVOC)관측초미세먼지,(BVOC)관측극초미세먼지,경과일,경과시간,관측미세먼지,관측초미세먼지
0,1.9,91.0,1026.1,0.9,328.8,1.600000,37.6,0.4,2.0,91.3,...,0.5,344.0,1.600000,12.5,12.6,12.4,31,0,38.5,38.3
1,1.0,88.9,1020.1,0.2,14.4,0.600000,37.3,0.4,0.9,89.1,...,0.4,24.1,0.900000,10.8,10.5,10.3,31,0,38.6,38.0
2,0.9,91.3,1017.6,0.8,4.2,2.000000,33.8,0.2,0.9,91.4,...,1.1,10.8,2.000000,12.6,12.0,11.8,31,0,35.4,34.5
6,-0.9,93.5,1026.2,0.0,0.0,1.225843,46.0,0.1,-0.8,93.4,...,0.0,0.0,1.316854,15.7,15.1,14.5,31,0,48.5,47.3
7,1.5,85.1,1024.7,0.0,0.0,0.500000,48.1,0.4,1.6,85.0,...,0.0,2.0,0.500000,15.6,14.9,14.6,31,0,51.2,49.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653446,10.2,63.2,1020.7,0.5,104.7,1.600000,3.1,0.4,10.1,63.0,...,0.4,40.0,1.600000,0.2,0.2,0.1,116,1270,4.2,3.7
653447,9.7,73.8,1016.9,0.4,179.3,1.100000,16.6,0.4,9.6,74.5,...,0.6,156.6,1.400000,12.7,9.3,8.8,116,1270,25.4,18.5
653448,9.3,74.5,1008.4,0.4,75.6,1.400000,18.2,0.3,9.3,74.5,...,0.5,96.8,1.400000,17.7,13.3,12.8,116,1270,27.2,19.7
653449,8.9,74.0,1002.8,0.3,282.8,0.600000,16.3,0.4,8.8,75.5,...,0.2,318.5,0.600000,10.8,9.2,9.0,116,1270,21.3,17.5


,관측온도,관측습도,관측기압,관측풍속,관측풍향,관측최대풍속,관측극초미세먼지,배관관측온도,(AVOC)관측온도,(AVOC)관측습도,...,(BVOC)관측풍속,(BVOC)관측풍향,(BVOC)관측최대풍속,(BVOC)관측미세먼지,(BVOC)관측초미세먼지,(BVOC)관측극초미세먼지,경과일,경과시간,관측미세먼지,관측초미세먼지
653539,8.2,66.0,1016.6,0.1,10.6,0.400000,6.1,0.4,8.4,64.5,...,0.2,21.8,0.500000,2.1,2.0,1.9,116,1280,7.0,6.5
653540,8.6,64.7,1014.0,1.0,34.2,2.300000,5.0,0.2,8.6,64.7,...,1.0,31.7,2.300000,1.7,1.6,1.6,116,1280,6.8,5.6
653544,7.1,87.0,1023.3,0.8,282.9,1.160465,13.8,0.2,7.1,87.3,...,0.8,279.4,1.256977,8.6,8.1,8.0,116,1280,15.3,14.5
653545,7.4,78.5,1021.9,0.3,76.3,0.800000,10.1,0.4,7.3,79.2,...,0.2,49.2,0.800000,6.6,5.3,4.9,116,1280,13.0,11.2
653546,8.0,79.1,1023.3,0.3,309.5,1.100000,7.1,0.0,8.0,79.5,...,0.3,261.8,1.700000,2.6,2.7,2.6,116,1280,7.9,7.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727187,10.0,47.3,1010.6,0.5,149.1,1.318605,13.6,0.3,10.0,47.3,...,0.5,92.5,1.402326,13.3,2.8,2.0,121,1430,56.3,21.9
727188,10.2,49.0,1010.4,0.3,134.3,0.900000,11.6,0.4,10.1,49.2,...,0.2,64.3,0.900000,1.1,0.0,0.0,121,1430,23.2,15.1
727189,7.2,59.7,1006.6,1.0,123.4,2.100000,12.7,0.4,7.0,60.0,...,0.8,127.8,2.100000,8.5,2.2,1.7,121,1430,47.1,19.6
727190,9.7,47.0,998.3,0.8,121.0,1.600000,13.8,0.3,9.6,46.8,...,0.7,123.8,1.600000,13.6,3.7,2.8,121,1430,53.6,21.0
